In [176]:
import geopandas as gpd
import pandas as pd

## Open Geojson

In [177]:
greece_gdf = gpd.read_file("data/periphereies/periphereies/periphereies.shp")

In [178]:
def fix_encoding(s):
    try:
        return s.encode("latin1").decode("windows-1253")
    except:
        return s


greece_gdf["PER"] = greece_gdf["PER"].apply(fix_encoding)

In [179]:
list(greece_gdf["PER"])

['Π. ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ - ΘΡΑΚΗΣ',
 'Π. ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ',
 'Π. ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ',
 'Π. ΗΠΕΙΡΟΥ',
 'Π. ΘΕΣΣΑΛΙΑΣ',
 'Π. ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ',
 'Π. ΝΟΤΙΟΥ ΑΙΓΑΙΟΥ',
 'Π. ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ',
 'Π. ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ',
 'Π. ΠΕΛΟΠΟΝΝΗΣΟΥ',
 'Π. ΙΟΝΙΩΝ ΝΗΣΩΝ',
 'Π. ΚΡΗΤΗΣ',
 'Π. ΑΤΤΙΚΗΣ']

In [180]:
mapping = {
    "Π. ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ - ΘΡΑΚΗΣ": "ΑΝΑΤΟΛΙΚΗ ΜΑΚΕΔΟΝΙΑ ΚΑΙ ΘΡΑΚΗ",
    "Π. ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ": "ΚΕΝΤΡΙΚΗ ΜΑΚΕΔΟΝΙΑ",
    "Π. ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ": "ΔΥΤΙΚΗ ΜΑΚΕΔΟΝΙΑ",
    "Π. ΗΠΕΙΡΟΥ": "ΗΠΕΙΡΟΣ",
    "Π. ΘΕΣΣΑΛΙΑΣ": "ΘΕΣΣΑΛΙΑ",
    "Π. ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ": "ΒΟΡΕΙΟ ΑΙΓΑΙΟ",
    "Π. ΝΟΤΙΟΥ ΑΙΓΑΙΟΥ": "ΝΟΤΙΟ ΑΙΓΑΙΟ",
    "Π. ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ": "ΣΤΕΡΕΑ ΕΛΛΑΔΑ",
    "Π. ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ": "ΔΥΤΙΚΗ ΕΛΛΑΔΑ",
    "Π. ΠΕΛΟΠΟΝΝΗΣΟΥ": "ΠΕΛΟΠΟΝΝΗΣΟΣ",
    "Π. ΙΟΝΙΩΝ ΝΗΣΩΝ": "ΙΟΝΙΟΙ ΝΗΣΟΙ",
    "Π. ΚΡΗΤΗΣ": "ΚΡΗΤΗ",
    "Π. ΑΤΤΙΚΗΣ": "ATTIKH",
}

In [181]:
greece_gdf["NAME"] = greece_gdf["PER"].map(mapping)

## Get Open Population data

In [182]:
population_df = pd.read_excel(
    "data/A1605_SPO18_TS_AN_00_2002_00_2024_10_F_BI.xlsx",
    sheet_name="ΣΥΝΟΛΟ-TOTAL_2014-2024",
    skiprows=3,
)

In [183]:
columns = list(population_df.iloc[23])
columns[0] = "NAME"
columns[-1] = "NAME_ENG"

In [184]:
population_df.columns = columns
population_df = population_df.iloc[24:88]
population_df = population_df[
    ~population_df["NAME"].str.startswith("Π.Ε.")
].reset_index(drop=True)

In [185]:
population_df["2014"] = population_df["1.1.2014"].astype(int)
population_df["2024"] = population_df["1.1.2024"].astype(int)
population_df["DIFF"] = population_df["2024"] - population_df["2014"]
population_df["PERCENT"] = (
    (population_df["DIFF"] / population_df["2014"]) * 100
).round(decimals=1)

In [186]:
population_df = population_df[["2014", "2024", "DIFF", "PERCENT", "NAME", "NAME_ENG"]]

## Merge data and export

In [187]:
greece_pop_gdf = greece_gdf.merge(population_df, on="NAME", how="left")

In [188]:
greece_pop_gdf.to_file("data/greece_pop.gpkg")

In [191]:
greece_pop_gdf

,PER,geometry,NAME,2014,2024,DIFF,PERCENT,NAME_ENG
0,Π. ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ - ΘΡΑΚΗΣ,"MULTIPOLYGON (((468363.189 4577898.998, 468365...",ΑΝΑΤΟΛΙΚΗ ΜΑΚΕΔΟΝΙΑ ΚΑΙ ΘΡΑΚΗ,608214,561834,-46380,-7.6,"ANATOLIKI MAKEDONIA, THRAKI"
1,Π. ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,"MULTIPOLYGON (((306365.379 4525842.998, 306368...",ΚΕΝΤΡΙΚΗ ΜΑΚΕΔΟΝΙΑ,1903360,1778896,-124464,-6.5,KENTRIKI MAKEDONIA
2,Π. ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,"POLYGON ((226087.989 4471633.998, 226102.009 4...",ΔΥΤΙΚΗ ΜΑΚΕΔΟΝΙΑ,278706,247270,-31436,-11.3,DYTIKI MAKEDONIA
3,Π. ΗΠΕΙΡΟΥ,"MULTIPOLYGON (((157581.777 4400827.998, 157583...",ΗΠΕΙΡΟΣ,341046,321108,-19938,-5.8,IPEIROS
4,Π. ΘΕΣΣΑΛΙΑΣ,"MULTIPOLYGON (((252882.049 4395793.998, 252884...",ΘΕΣΣΑΛΙΑ,737686,678747,-58939,-8.0,THESSALIA
5,Π. ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,"MULTIPOLYGON (((583302.439 4372793.498, 583308...",ΒΟΡΕΙΟ ΑΙΓΑΙΟ,198581,201007,2426,1.2,VOREIO AIGAIO
6,Π. ΝΟΤΙΟΥ ΑΙΓΑΙΟΥ,"MULTIPOLYGON (((511410.309 4039985.999, 511410...",ΝΟΤΙΟ ΑΙΓΑΙΟ,334802,327246,-7556,-2.3,NOTIO AIGAIO
7,Π. ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ,"MULTIPOLYGON (((272558.249 4321865.998, 272571...",ΣΤΕΡΕΑ ΕΛΛΑΔΑ,559214,505073,-54141,-9.7,STEREA ELLADA
8,Π. ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ,"MULTIPOLYGON (((245118.969 4192904.499, 245147...",ΔΥΤΙΚΗ ΕΛΛΑΔΑ,677727,640243,-37484,-5.5,DYTIKI ELLADA
9,Π. ΠΕΛΟΠΟΝΝΗΣΟΥ,"MULTIPOLYGON (((281006.999 4103543.999, 281008...",ΠΕΛΟΠΟΝΝΗΣΟΣ,585155,531598,-53557,-9.2,PELOPONNISOS


In [192]:
greece_pop_gdf["2014"].sum()

np.int64(10926807)

In [193]:
greece_pop_gdf["2024"].sum()

np.int64(10400720)